# PerceptionNode Yapısı (.hpp) Açıklaması

Bu not defteri, `perception_node.hpp` dosyasında tanımlanan **PerceptionNode** sınıfının yapısını, amacını ve fonksiyonlarını adım adım açıklamak için hazırlanmıştır.

Genel amaç:

- Algılama (perception) ile ilgili tüm işlemleri **tek bir C++ sınıfı** içinde toplamak,
- Dışarıya sade bir API sunmak: `perception.process(frame)`,
- Model yükleme, preprocess, inference ve postprocess gibi ayrıntıları sınıf içinde soyutlamak.

## 1. Header Guard ve Dahil Edilen Dosyalar

```cpp
#ifndef PERCEPTION_NODE_HPP
#define PERCEPTION_NODE_HPP

#include "perception_types.hpp"

#include <torch/script.h>   // torch::jit::script::Module, torch::Tensor, torch::Device
#include <vector>
#include <string>
```
- `#ifndef / #define / #endif`: Header guard, bu dosyanın içeriğinin **sadece bir kez** derlenmesini sağlar.
- `#include "perception_types.hpp"`: `ImageFrame`, `DetectedObject`, `PerceptionConfig` gibi veri tiplerini kullanmak için dahil edilir.
- `#include <torch/script.h>`: LibTorch TorchScript API'si (model yükleme, tensor, device için).
- `<vector>` ve `<string>`: Standart C++ kapsayıcı ve string tipleri için.

`#endif // PERCEPTION_NODE_HPP` satırı dosyanın sonunda yer alır ve header guard bloğunu kapatır.

## 2. PerceptionNode Sınıfının Genel Tanımı

```cpp

class PerceptionNode
{
public:
    explicit PerceptionNode(const PerceptionConfig& config);

    std::vector<DetectedObject> process(const ImageFrame& frame);

private:
    void loadModel(const std::string& path);

    torch::Tensor preprocess(const ImageFrame& frame);
    torch::Tensor inference(const torch::Tensor& input);
    std::vector<DetectedObject> postprocess(const ImageFrame& frame,
                                            const torch::Tensor& output);

private:
    PerceptionConfig config_;
    torch::jit::script::Module model_;
    torch::Device device_{torch::kCPU};
    bool model_loaded_{false};
};
```

Bu sınıf:

- **Input**: `ImageFrame` (tek bir kamera karesi + bağlam bilgisi),
- **Output**: `std::vector<DetectedObject>` (tespit edilen nesne listesi) üretir.

Dışarıdan sadece `process()` çağrılır; tüm detaylar (model yükleme, preprocess, inference, postprocess) sınıf içinde kapsüllenmiştir.

## 3. Constructor – `PerceptionNode(const PerceptionConfig& config)`

```cpp
explicit PerceptionNode(const PerceptionConfig& config);
```

- `explicit`: Tek argümanlı constructor'ın **örtük (implicit) dönüşüm** için kullanılmasını engeller. Yani:
  - `PerceptionNode node(config);` geçerlidir,
  - `PerceptionNode node = config;` gibi hatalı kullanımlar engellenir.
- `const PerceptionConfig& config`: Node'un çalışma şeklini belirleyen **ayar paketi** referans olarak alınır.
  - `model_path`, `device`, `input_width/height`, `mean/std`, `class_names` gibi tüm değerler buradan okunur.

Constructor'ın tipik görevleri:

1. `config_` üyesini doldurmak,
2. Cihaz seçimini yapmak (CPU / CUDA),
3. `model_path` boş değilse TorchScript modeli yüklemek (Uygulama 1'de bu adım opsiyoneldir).

## 4. Ana API – `process(const ImageFrame& frame)`

```cpp
std::vector<DetectedObject> process(const ImageFrame& frame);
```

Bu fonksiyon, dış dünyaya açılan **tek ana arabirimdir**.

Akış sırası:

1. `preprocess(frame)`  
   - `ImageFrame` içindeki `cv::Mat` görüntü alınır,
   - Gerekli `resize` / `normalize` işlemleri yapılarak `torch::Tensor`'a dönüştürülür.

2. `inference(input_tensor)`  
   - TorchScript model üzerinde `forward` çağrısı yapılır,
   - Model çıktısı `torch::Tensor` olarak döner.

3. `postprocess(frame, output_tensor)`  
   - Model çıktısı decode edilir,
   - BBox, class_id, skor gibi bilgiler hesaplanır,
   - `DetectedObject` listesi oluşturulur.

**Uygulama 1'de:** Bu fonksiyonun amacı sadece iskeleti tanımak; preprocess/inference/postprocess içeride dummy (sahte) log ve örnekler ile çalıştırılır.

## 5. Yardımcı Fonksiyonlar

### 5.1. `loadModel(const std::string& path)`

```cpp
void loadModel(const std::string& path);
```

- Görevi:
  - Verilen `path` üzerinden TorchScript modelini yüklemek,
  - Modeli `device_` üzerinde hazır hale getirmek (CPU veya CUDA),
  - `model_loaded_` bayrağını güncellemek.

Kullanım:

- Constructor içinde, `config_.model_path` boş değilse çağrılır.
- Model yüklenemezse hata log'lanır ve `model_loaded_ = false` kalır.

---

### 5.2. `preprocess(const ImageFrame& frame)`

```cpp
torch::Tensor preprocess(const ImageFrame& frame);
```

- Görevi:
  - `frame.image` (cv::Mat) üzerinde:
    - `resize` (config.input_width / input_height),
    - BGR → RGB dönüşümü,
    - float'a çevirme ve [0, 1] aralığına normalizasyon,
    - `mean` ve `std` ile kanal bazlı normalizasyon,
    - NCHW (batch, channel, height, width) formatına dönüştürme
  - Sonuç: `torch::Tensor`

**Uygulama 1'de:** Sadece dummy bir tensor döndürür ve log yazar.

---

### 5.3. `inference(const torch::Tensor& input)`

```cpp
torch::Tensor inference(const torch::Tensor& input);
```

- Görevi:
  - `model_` üzerinde `forward` çalıştırmak,
  - `torch::NoGradGuard` ile gradient hesaplamayı kapatmak,
  - Modelin ham çıktısını `torch::Tensor` olarak döndürmek.

**Uygulama 1'de:** Eğer `model_loaded_` false ise dummy bir tensor döner, sadece akışın iskeletini göstermek için kullanılır.

---

### 5.4. `postprocess(const ImageFrame& frame, const torch::Tensor& output)`

```cpp
std::vector<DetectedObject> postprocess(const ImageFrame& frame,
                                        const torch::Tensor& output);
```

- Görevi:
  - Model çıktısını decode etmek,
  - Her tespit için `DetectedObject` oluşturmak,
  - `class_id`, `class_name`, `score`, bbox (`x, y, width, height`) gibi alanları doldurmak,
  - Gerekirse NMS (Non-Maximum Suppression) uygulamak,
  - Sonuçları `std::vector<DetectedObject>` olarak döndürmek.

**Uygulama 1'de:** Sadece 1 adet sahte (dummy) `DetectedObject` oluşturur; iskelet akışını görmek için kullanılır.

## 6. Üye Değişkenler

```cpp
private:
    PerceptionConfig config_;
    torch::jit::script::Module model_;
    torch::Device device_{torch::kCPU};
    bool model_loaded_{false};
```

### 6.1. `config_`

- `PerceptionConfig` tipinde, node'un çalışma ayarlarını tutar.
- Model yolu, cihaz, input boyutu, mean/std, sınıf isimleri gibi tüm bilgiler burada saklanır.

### 6.2. `model_`

- TorchScript model nesnesidir.
- `loadModel()` içinde `torch::jit::load()` ile yüklenecek.
- `inference()` fonksiyonu bu nesne üzerinden `forward` çağrısı yapar.

### 6.3. `device_`

- Kullanılacak cihazı temsil eder: `torch::kCPU` veya `torch::kCUDA`.
- Constructor içinde `config_.device` ve `torch::cuda::is_available()` kontrolü ile belirlenir.
- Hem model hem de tensor'ler bu device üzerinde tutulur.

### 6.4. `model_loaded_`

- Modelin gerçekten başarıyla yüklenip yüklenmediğini takip eden bayraktır.
  - `true` → model hazır, inference yapılabilir.
  - `false` → model yok veya yükleme hatası oluşmuş; inference fonksiyonu dummy davranmalıdır.

Bu bayrak sayesinde, model yüklenmemişken `forward` çağırıp programın çökmesi engellenir.

## 7. Özet

`PerceptionNode`, algılama pipeline'ını temiz ve modüler bir sınıf yapısına oturtmak için tasarlanmıştır:

- **Dış API**: `process(const ImageFrame&)`  
  → Algılama ile ilgili tüm adımlar tek fonksiyon çağrısına indirgenir.

- **İç Adımlar**:
  - `preprocess()` → cv::Mat → torch::Tensor
  - `inference()`  → model.forward(tensor)
  - `postprocess()` → model çıktısından `DetectedObject` listesi

- **Konfigürasyon**:
  - `PerceptionConfig` ile model yolu, device, input size, normalizasyon ve sınıf isimleri dışarıdan belirlenir.

Bu yapı sayesinde:

- Algılama kodu "fonksiyon yığını" olmaktan çıkar,
- Gerçek zamanlı sistemlerde node mantığına uygun, bakımı kolay ve test edilebilir bir modül haline gelir.


---
---
----

# `perception_node.cpp` Yapısı ve Akış Açıklaması

Bu not defteri, **Uygulama 1** için yazdığımız `perception_node.cpp` dosyasının tamamını adım adım açıklamak için hazırlanmıştır.

Amaç:

- `PerceptionNode` sınıfının **.cpp tarafındaki gerçek implementasyonunu** anlamak,
- Constructor, `loadModel`, `preprocess`, `inference`, `postprocess` ve `process` fonksiyonlarının rollerini netleştirmek,
- Bu aşamada neden **dummy (sahte)** bir pipeline kullandığımızı görmek.

## 1. Dosya Başlangıcı – `#include` Satırları

```cpp
#include "perception_node.hpp"
#include <iostream>
```

### Açıklama

- `#include "perception_node.hpp"`  
  - Sınıfın deklarasyonunun (tanım imzasının) bulunduğu header dosyayı projeye dahil eder.  
  - Burada:
    - `PerceptionNode` sınıfı,
    - Constructor imzası,
    - `process`, `preprocess`, `inference`, `postprocess`, `loadModel` fonksiyon imzaları,
    - Üye değişkenler (`config_`, `model_`, `device_`, `model_loaded_`) tanımlıdır.

- `#include <iostream>`  
  - `std::cout` ve `std::cerr` kullanarak log ve hata mesajı yazdırmak için gereklidir.

Bu sayede `.cpp` tarafında:

- Hem sınıfın üye fonksiyonlarını tanımlayabiliriz,
- Hem de konsola debug/log çıktıları gönderebiliriz.

## 2. Constructor – `PerceptionNode::PerceptionNode`

```cpp
PerceptionNode::PerceptionNode(const PerceptionConfig& config)
    : config_(config)
{
    std::cout << "[PerceptionNode] Node oluşturuluyor...\n";

    // Cihaz seçimi
    if (config_.device == "cuda" && torch::cuda::is_available())
    {
        device_ = torch::kCUDA;
        std::cout << "[PerceptionNode] CUDA kullanılacak.\n";
    }
    else
    {
        device_ = torch::kCPU;
        std::cout << "[PerceptionNode] CPU kullanılacak.\n";
    }

    // Model yükleme
    if (!config_.model_path.empty())
    {
        std::cout << "[PerceptionNode] Model yükleniyor: "
                  << config_.model_path << "\n";
        loadModel(config_.model_path);
    }
    else
    {
        std::cout << "[PerceptionNode] Model path boş, dummy mod aktif.\n";
        model_loaded_ = false;
    }
}
```

### Ne yapıyor?

1. **Üye kopyalama:**  
   ```cpp
   : config_(config)
   ```  
   - Constructor parametresi olarak gelen `config` nesnesi, sınıfın içindeki `config_` üyesine kopyalanır.
   - Bu sayede node, tüm çalışma ayarlarını (`model_path`, `device`, `input_width/height`, `mean/std`, `class_names`) kendi içinde saklar.

2. **Cihaz seçimi (CPU / CUDA):**  
   ```cpp
   if (config_.device == "cuda" && torch::cuda::is_available())
   {
       device_ = torch::kCUDA;
   }
   else
   {
       device_ = torch::kCPU;
   }
   ```  
   - `config_.device == "cuda"` → Kullanıcı config seviyesinde "cuda" istedi mi?
   - `torch::cuda::is_available()` → Ortamda CUDA gerçekten kullanılabilir mi?  
   - İki koşul da sağlanırsa:
     - `device_ = torch::kCUDA;` → node GPU üzerinde çalışacak şekilde hazırlanır.
   - Aksi halde:
     - `device_ = torch::kCPU;` → CPU üzerinde çalışılacaktır.

3. **Model yükleme kararı:**  
   ```cpp
   if (!config_.model_path.empty())
   {
       loadModel(config_.model_path);
   }
   else
   {
       model_loaded_ = false;
   }
   ```  
   - Eğer `model_path` boş değilse, `loadModel()` çağrılır ve TorchScript modelini yüklemeye çalışır.
   - Eğer `model_path` boşsa, `model_loaded_ = false` kalır ve node **dummy modda** çalışır.

### Uygulama 1 açısından

Bu aşamada:
- Asıl amaç iskeletin ayağa kalktığını görmek.
- Gerçek model kullanmak zorunda değiliz; path boş bırakılabilir.
- Constructor, sadece:
  - Cihaz seçimi,
  - Dummy mod bilgisi,
  - Basit log mesajları üretir.

## 3. `loadModel` Fonksiyonu

```cpp
void PerceptionNode::loadModel(const std::string& path)
{
    try
    {
        model_ = torch::jit::load(path);
        model_.to(device_);
        model_loaded_ = true;

        std::cout << "[PerceptionNode] Model başarıyla yüklendi.\n";
    }
    catch (const std::exception& e)
    {
        std::cerr << "[PerceptionNode] Model yüklenemedi: " << e.what() << "\n";
        model_loaded_ = false;
    }
}
```

### Ne yapıyor?

- Verilen `path` üzerinden TorchScript modelini yüklemeye çalışır:
  - `torch::jit::load(path)` ile `model_` nesnesine yükler.
  - `model_.to(device_);` ile modeli CPU veya CUDA cihazına taşır.
- Eğer her şey başarılıysa:
  - `model_loaded_ = true;`
  - Başarılı log mesajı basar.
- Hata olursa (`try/catch` bloğu):
  - Hata mesajını `std::cerr` ile ekrana yazar,
  - `model_loaded_ = false;` olarak işaretler.

### Uygulama 1 açısından

- `model_path` boş bırakıldığı için genellikle Uygulama 1'de buraya hiç girmez.
- Yine de iskelet düzgün olsun diye gerçek bir `loadModel` fonksiyonu tanımlanmıştır.

## 4. `preprocess` Fonksiyonu (Uygulama 1 — Dummy)

```cpp
torch::Tensor PerceptionNode::preprocess(const ImageFrame& frame)
{
    std::cout << "[preprocess] Frame alındı. Boyut: "
              << frame.image.cols << "x" << frame.image.rows << "\n";

    std::cout << "[preprocess] Uygulama 1 modunda çalışıyor: dummy tensor döndürülüyor.\n";

    return torch::rand({1, 3, 640, 640});
}
```

### Ne yapıyor?

- `frame.image` içindeki görüntünün boyutunu log olarak yazdırıyor.
- Gerçek bir preprocess pipeline'ı (resize, normalize, tensor formatı) henüz uygulamıyor.
- Bunun yerine:
  - `torch::rand({1, 3, 640, 640})` ile rastgele değerlerden oluşan bir tensor döndürüyor.

Bu tensor:
- Shape: `[1, 3, 640, 640]` → (batch=1, channel=3, height=640, width=640)
- Değerler: 0–1 aralığında rastgele float değerler.

### Neden böyle?

Uygulama 1'in amacı:

> “Algılama node iskeleti doğru akıyor mu?”

Bu aşamada:
- Model yok,
- Gerçek preprocess yok,
- Sadece `process()` içindeki çağrı zincirini görmek istiyoruz.

## 5. `inference` Fonksiyonu (Uygulama 1 — Dummy)

```cpp
torch::Tensor PerceptionNode::inference(const torch::Tensor& input)
{
    std::cout << "[inference] Girilen tensor shape: " << input.sizes() << "\n";

    if (!model_loaded_)
    {
        std::cout << "[inference] Model yüklenmemiş. Dummy output döndürülüyor.\n";
        return torch::rand({1, 1});
    }

    torch::NoGradGuard no_grad;

    auto output = model_.forward({input}).toTensor();

    std::cout << "[inference] Model çıktısı alındı.\n";

    return output;
}
```

### Ne yapıyor?

1. **Girdi tensor'ünün boyutunu loglar:**
   ```cpp
   std::cout << "[inference] Girilen tensor shape: " << input.sizes() << "\n";
   ```

2. **Model yüklü mü kontrol eder:**
   ```cpp
   if (!model_loaded_)
   {
       // Dummy mod
       return torch::rand({1, 1});
   }
   ```
   - Eğer model yüklü değilse (`model_loaded_ == false`):
     - Rastgele bir tensor (`[1, 1]` boyutunda) döndürür.
     - Bu, Uygulama 1'de **dummy output** olarak kullanılır.

3. **Model yüklü ise gerçek forward yapar:**
   ```cpp
   torch::NoGradGuard no_grad;
   auto output = model_.forward({input}).toTensor();
   ```
   - `torch::NoGradGuard`: Inference sırasında gradient hesaplamasını kapatır.
   - `model_.forward({input})`: TorchScript modeline forward çağrısı yapar.
   - `.toTensor()`: Çıktıyı tensor'e çevirir.

### Uygulama 1 açısından

- Çoğu durumda dummy modda çalışır (`model_loaded_ = false`).
- Ama iskelet düzgün olduğunda Uygulama 2–3–4 aşamalarında gerçek model forward'ını buraya koymamız çok kolay olur.

## 6. `postprocess` Fonksiyonu (Uygulama 1 — Dummy)

```cpp
std::vector<DetectedObject> PerceptionNode::postprocess(
    const ImageFrame& frame,
    const torch::Tensor& output)
{
    std::cout << "[postprocess] Dummy postprocess çalışıyor. output shape: "
              << output.sizes() << "\n";

    // Tek adet sahte nesne
    DetectedObject obj;
    obj.class_id = 0;
    obj.class_name = "dummy_object";
    obj.score = 0.9f;
    obj.x = 100;
    obj.y = 120;
    obj.width = 50;
    obj.height = 60;

    return {obj};
}
```

### Ne yapıyor?

- `output` tensor'ünün şeklini loglar.
- Gerçek bir decode / NMS / bbox hesaplama yapmaz.
- Bunun yerine:
  - Tek bir `DetectedObject` nesnesi oluşturur,
  - İçine sahte bir tespit yazar:
    - `class_id = 0`
    - `class_name = "dummy_object"`
    - `score = 0.9`
    - `bbox = (100, 120, 50, 60)`

- Sonuçta bu `DetectedObject`'i içeren bir `std::vector` döndürür.

### Uygulama 1 açısından

Uygulama 1'in amacı tespitin **gerçekçi** olması değil, sadece şunları görmektir:

- `process()` fonksiyonu bir `std::vector<DetectedObject>` döndürüyor mu?
- `main.cpp` içinden bu vektörü düzgün okuyup loglayabiliyor muyuz?
- `DetectedObject` struct'ının alanları (`class_id`, `class_name`, `score`, `bbox`, `track_id`) gerçek hayatta nasıl görünecek?

## 7. `process` Fonksiyonu — Ana Pipeline

```cpp
std::vector<DetectedObject> PerceptionNode::process(const ImageFrame& frame)
{
    std::cout << "[process] === Algılama pipeline başlatıldı ===\n";

    auto input_tensor = preprocess(frame);
    auto output = inference(input_tensor);
    auto detections = postprocess(frame, output);

    std::cout << "[process] === Pipeline tamamlandı ===\n";

    return detections;
}
```

### Ne yapıyor?

Bu fonksiyon, dış dünyaya açılan **ana API**'dir:

1. **Başlangıç log'u:**
   ```cpp
   std::cout << "[process] === Algılama pipeline başlatıldı ===\n";
   ```

2. **Preprocess çağrısı:**
   ```cpp
   auto input_tensor = preprocess(frame);
   ```
   - `ImageFrame` → `torch::Tensor`
   - Uygulama 1'de dummy tensor.

3. **Inference çağrısı:**
   ```cpp
   auto output = inference(input_tensor);
   ```
   - Preprocess'ten gelen tensor → model / dummy inference.

4. **Postprocess çağrısı:**
   ```cpp
   auto detections = postprocess(frame, output);
   ```
   - Modelin raw çıktısı → `std::vector<DetectedObject>`

5. **Bitiş log'u ve dönüş:**
   ```cpp
   std::cout << "[process] === Pipeline tamamlandı ===\n";
   return detections;
   ```

### Neden bu kadar önemli?

Çünkü dışarıdan bakan kod için tek gördüğümüz şudur:

```cpp
auto detections = perception.process(frame);
```

İçeride:
- Kaç adım var,
- Hangi tensor'ler dönüyor,
- Model mi var dummy mi,  

bunların hiçbiri dışarıyı ilgilendirmez.

Bu, algoritmayı **modüler, test edilebilir ve değiştirmesi kolay** hale getirir.

## 8. Genel Özet

`perception_node.cpp`, `PerceptionNode` sınıfının **gerçek implementasyonunu** içerir ve Uygulama 1 için şu şekilde çalışır:

- Constructor:
  - Config'i içeri alır,
  - CPU / CUDA cihazını seçer,
  - Gerekirse modeli yüklemeyi dener,
  - Aksi halde dummy modda çalışır.

- `preprocess`:
  - Şimdilik sadece log basar ve rastgele bir tensor döndürür.

- `inference`:
  - Model yoksa dummy output üretir,
  - Model varsa gerçek `forward` çağrısını yapacak yapıda tasarlanmıştır.

- `postprocess`:
  - Tek sahte `DetectedObject` oluşturur ve geri döndürür.

- `process`:
  - Tüm pipeline'ı (preprocess → inference → postprocess) tek bir fonksiyon altında toplar.

Uygulama 1'de amaç:

> "Gerçek algılama kalitesi" değil,  
> "Node mimarisinin, sınıf yapısının ve veri akışının doğru oturmasıdır."

Uygulama 2 ve sonrasında:

- `preprocess` gerçek hale getirilecek,
- Daha sonra `inference` gerçek TorchScript modeline bağlanacak,
- Son aşamada `postprocess` ile gerçek bbox & class decode + NMS yazılacaktır.
